In [1]:
import sympy as sp
import numpy as np

In [2]:
def finite_difference_coefficients(stencil, derivative_order):
    """
    Given a stencil (list of points, e.g. [-2,-1,0,1,2]) and an integer derivative_order n,
    return a tuple (coeffs, accuracy) where coeffs is a list of sympy expressions representing the 
    coefficients for approximating f^(n)(0) with:
    
        f^(n)(0) ~ (1/h^n) * sum_i coeffs[i] * f(h*stencil[i])
    
    and accuracy is the order (i.e. the leading error term is O(h^(accuracy))).
    
    Raises a ValueError if the stencil is not large enough (i.e. if len(stencil) < derivative_order+1).
    """
    m = len(stencil)
    if m < derivative_order + 1:
        raise ValueError("Stencil is not large enough to approximate derivative order %d" % derivative_order)
    
    # Define symbols for the unknown weights: c0, c1, ..., c_{m-1}
    c = sp.symbols('c0:%d' % m)
    
    # Set up the moment equations:
    # For j = 0,1,...,m-1, we require:
    #     sum_i c_i*(s_i)^j = (j! if j==derivative_order else 0)
    eqs = []
    for j in range(m):
        eq = sp.Eq(sum(c[i] * (stencil[i])**j for i in range(m)),
                   (sp.factorial(j) if j == derivative_order else 0))
        eqs.append(eq)
    
    # Solve the system of equations for the coefficients
    sol = sp.solve(eqs, c, dict=True)
    if not sol:
        raise ValueError("Could not solve for finite difference weights. Check the stencil.")
    sol = sol[0]
    
    # Extract and simplify the coefficients
    coeffs = [sp.simplify(sol[c[i]]) for i in range(m)]
    
    # Determine the achieved polynomial accuracy:
    # We find the smallest k >= m for which:
    #   (1/k!)*sum_i c_i*(s_i)^k != (1 if k==derivative_order else 0)
    k = m
    while True:
        moment = sum(sol[c[i]] * (stencil[i])**k for i in range(m)) / sp.factorial(k)
        expected = 1 if k == derivative_order else 0
        if sp.simplify(moment - expected) != 0:
            break
        k += 1
    accuracy = k - derivative_order
    return coeffs, accuracy


In [3]:
# Example usage:

# A symmetric 5-point stencil, the point at which we evaluate is at 0
stencil = [-2, -1, 0, 1, 2]
# we want the second derivative:
n = 2

# Compute the finite difference coefficients and print the result
coeffs, acc = finite_difference_coefficients(stencil, n)
sp.init_printing()
print("Finite difference coefficients for f^(%d)(0):" % n)
sp.pprint(coeffs)
print("\nAchieved accuracy: O(h^(%d))" % acc)

Finite difference coefficients for f^(2)(0):
[-1/12, 4/3, -5/2, 4/3, -1/12]

Achieved accuracy: O(h^(4))
